In [ ]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
#import psycopg2

# from config import db_password

import time

In [ ]:
# 1. Add the clean movie function that takes in the argument, "movie".

# Add function from this module for clean movie
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    return movie
    
    
    

 

In [ ]:
def parse_dollars(s):
    if type(s) != str:
        return np.nan
    if s == '':
        return np.nan
    s = s.replace('$', '').replace(',','').lower()
    # if input is of the form ###.# mil
    p = re.compile(r'\d+\.?\d*\s*mil')
    match = re.search(p, s)
    if match:
        total_box_office = match.group(0).replace('mil', '').rstrip()
        value = float(total_box_office) * 10**6
        return value
    p = re.compile(r'\d+\.?\d*\s*bil')
    match = re.search(p, s)
    if match:
        total_box_office = match.group().replace('bil', '').rstrip()
        value = float(total_box_office) * 10**9
        return value
    p = re.compile(r'\d+')
    match = re.search(p, s)
    if match:
        total_box_office = match.group()
        value = float(total_box_office)
        return value
    return np.nan

In [ ]:
def parse_dates(s):
    if type(s) != str:
        return np.nan
    if s == '':
        return np.nan
    s = s.replace('$', '').replace(',','').lower()
    # if input is of the form ###.# mil
   
    p = re.compile(r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}')
    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value
    
    p = re.compile(r'\d+\.?\d*\s*bil')
    match = re.search(p, s)
    if match:
        value = match.group(0)
        return value
   
    p = re.compile(r'\d{4}.[01]\d.[123]\d')
    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value
     
    p = re.compile (r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}')
    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value
     
    p = re.compile (r'\d{4}')

    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value

    return np.nan

In [ ]:
def parse_times(s):
    if type(s) != str:
        return np.nan
    if s == '':
        return np.nan
    p = re.compile(r'\d* min')
    match = re.search(p, s)
    if match:
        value = match.group(0)
        value = value.replace(' min', '')
        #value = int(value)
        return value
    return np.nan

In [ ]:
file_dir = 'C:/users/lrosa/OneDrive/Desktop/Module 8'
kaggle_metadata=(f"{file_dir}/movies_metadata.csv")

ratings=(f"{file_dir}/ratings.csv")

wiki_movies_raw=(f'{file_dir}/wikipedia-movies.json') 

In [ ]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)


 #Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.


# Open and read the Wikipedia data JSON file.

In [ ]:
def extract_transform_load(kaggle_metadata,ratings,wiki_movies_raw):
    kaggle_metadata=pd.read_csv(kaggle_metadata,low_memory=False)

    ratings=pd.read_csv(ratings,low_memory=False)

    with open(wiki_movies_raw, mode='r') as file:
        wiki_movies_raw = json.load(file)





    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies=[movie for movie in wiki_movies_raw if("Director" in movie or "Directed by" in movie) \
             and "imdb_link" in movie and "No. of episodes" not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies=[clean_movie(movie)for movie in wiki_movies]
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df=pd.DataFrame(clean_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    # extract IMDb ID
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    cols=[column for column in wiki_movies_df.columns if wiki_movies_df[column].count()>(wiki_movies_df.shape[0]/2)]
    wiki_movies_df=wiki_movies_df[cols]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office=wiki_movies_df["Box office"].dropna()

    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    r'\$\d+\.?\d*\s*mil.*'
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    # 12. Add the parse_dollars function.
    box_office=box_office.apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    wiki_movies_df['Box office'] = box_office
    wiki_movies_df['Box office']
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    Budget_office=wiki_movies_df["Budget"].dropna()
    Budget_office = Budget_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
    Budget_office=Budget_office.apply(parse_dollars)
    wiki_movies_df.drop("Budget",axis=1, inplace= True)
    wiki_movies_df["Budget"]=Budget_office
    # 15. Clean the release date column in the wiki_movies_df DataFrame.


    Release=wiki_movies_df["Release date"].dropna()
    Release = Release.apply(lambda x: ' '.join(x) if type(x) == list else x)
    Release =Release.apply(parse_dates)
    wiki_movies_df.drop("Release date",axis=1, inplace= True)
    wiki_movies_df["Release date"]=Release
    wiki_movies_df
    # 16. Clean the running time column in the wiki_movies_df DataFrame.


    #16 Clean the running time column in the wiki_movies_df DataFrame
    running_time = wiki_movies_df['Running time'].dropna()
    running_time = running_time.apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time = running_time.apply(parse_times)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    wiki_movies_df['Running time'] = running_time
    
    return kaggle_metadata,ratings,wiki_movies_df

In [ ]:
file_dir = 'Resources'
wiki_file = f'{file_dir}/wikipedia-movies.json'
kaggle_file = f'{file_dir}/movies_metadata.csv'
ratings_file = f'{file_dir}/ratings.csv'
wiki_df, kaggle_df, ratings_df = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [ ]:
wiki_movies_df

In [ ]:
# 13. Clean the box office column in the wiki_movies_df DataFrame.



In [ ]:


# Return three variables. The first is the wiki_movies_df DataFrame

#

In [ ]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df

In [ ]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()